# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

# ЧАСТЬ 3

### Пришлось разбить на несколько файлов, чтобы иметь возможность обучать параллельно несколько заданий.

In [363]:
#!g1.1
import random

import numpy as np
import scipy
from itertools import combinations

from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [364]:
#!g1.1
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

Using TensorFlow backend.


4423680/4422102 [==============================] - 3s 1us/step


__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [372]:
#!c1.8
from sklearn.base import BaseEstimator, TransformerMixin

class RFFPipeline_Sign(BaseEstimator, TransformerMixin): 
    def __init__(self, model, n_features=1000, new_dim=50, use_PCA=True):
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.normalizer = Normalizer()
        self.new_dim = new_dim
        self.PCA_model = PCA(new_dim)
        self.model = model

    def fit(self, X, y):
        X = self.normalizer.fit_transform(X)
        if self.use_PCA:
            X = self.PCA_model.fit_transform(X, y)
            
        elements_for_choosing = random.sample(range(X.shape[0]), 1420) # выбираю случ. элементы из выборки для составления пар
        
        dist_list = [] # список пар
        for (c,p) in combinations(elements_for_choosing, 2): # составляю комбинации пар выборки
            dist_list.append(list(scipy.spatial.distance.cdist(X[c].reshape(1,-1), 
                                                               X[p].reshape(1,-1),
                                                               lambda u, v: ((u-v)**2).sum())[0]))
        self.median = np.median(dist_list) # рассчитываю медиану
        self.w = np.random.normal(scale=1./self.median, size=(X.shape[1], self.n_features)) # генерирую набор весов
        self.b = np.random.uniform(-np.pi, np.pi, size=self.n_features) # генерирую набор свдигов
        
        new_features_X = np.sign(np.dot(X, self.w))# + self.b)
        self.model.fit(new_features_X, y)
        return
    
    def predict_proba(self, X):
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.sign(np.dot(X, self.w))# + self.b)
        return self.model.predict_proba(new_features_X)
        
    def predict(self, X):
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.sign(np.dot(X, self.w))# + self.b)
        return self.model.predict(new_features_X)

In [373]:
#!c1.8
forest = RandomForestClassifier(random_state = 72)
forest_rffp_kernel = RFFPipeline_Sign(model=forest)
forest_rffp_kernel.fit(x_train, y_train)
forest_preds = forest_rffp_kernel.predict(x_test)
print('SVM с rbf-ядром на случайных признаках. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(forest_preds, y_test)))

SVM с rbf-ядром на случайных признаках. Доля верных ответов на тестовой выборке: 0.86610



In [ ]:
#!c1.8
from sklearn.base import BaseEstimator, TransformerMixin

class RFFPipeline_Sign(BaseEstimator, TransformerMixin): 
    def __init__(self, model, n_features=1000, new_dim=50, use_PCA=True):
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.normalizer = Normalizer()
        self.new_dim = new_dim
        self.PCA_model = PCA(new_dim)
        self.model = model

    def fit(self, X, y):
        X = self.normalizer.fit_transform(X)
        if self.use_PCA:
            X = self.PCA_model.fit_transform(X, y)
            
        elements_for_choosing = random.sample(range(X.shape[0]), 1420) # выбираю случ. элементы из выборки для составления пар
        
        dist_list = [] # список пар
        for (c,p) in combinations(elements_for_choosing, 2): # составляю комбинации пар выборки
            dist_list.append(list(scipy.spatial.distance.cdist(X[c].reshape(1,-1), 
                                                               X[p].reshape(1,-1),
                                                               lambda u, v: ((u-v)**2).sum())[0]))
        self.median = np.median(dist_list) # рассчитываю медиану
        self.w = np.random.normal(scale=1./self.median, size=(X.shape[1], self.n_features)) # генерирую набор весов
        self.b = np.random.uniform(-np.pi, np.pi, size=self.n_features) # генерирую набор свдигов
        
        new_features_X = np.sign(np.dot(X, self.w) + self.b)
        self.model.fit(new_features_X, y)
        return
    
    def predict_proba(self, X):
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.sign(np.dot(X, self.w) + self.b)
        return self.model.predict_proba(new_features_X)
        
    def predict(self, X):
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.sign(np.dot(X, self.w) + self.b)
        return self.model.predict(new_features_X)

In [374]:
#!c1.8
forest = RandomForestClassifier(random_state = 72)
forest_rffp_kernel = RFFPipeline_Sign(model=forest)
forest_rffp_kernel.fit(x_train, y_train)
forest_preds = forest_rffp_kernel.predict(x_test)
print('SVM с rbf-ядром на случайных признаках. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(forest_preds, y_test)))

SVM с rbf-ядром на случайных признаках. Доля верных ответов на тестовой выборке: 0.86460



In [375]:
#!c1.8
forest = SVC(kernel='linear', random_state = 72)
forest_rffp_kernel = RFFPipeline_Sign(model=forest)
forest_rffp_kernel.fit(x_train, y_train)
forest_preds = forest_rffp_kernel.predict(x_test)
print('SVM с rbf-ядром на случайных признаках. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(forest_preds, y_test)))

SVM с rbf-ядром на случайных признаках. Доля верных ответов на тестовой выборке: 0.82940



**Вывод:** Поэкспериментировал со знаком от скалярного произведения в качестве случайных признаков, а также со случайным лесом в качестве альтернативного классификатора. Лучшего качества, к сожалению, добиться не удалось. Была мысль ещё попробовать функцию Лапласса, но реализовать не успел.